In [ ]:
import os
import numpy as np
import shutil
import json
import pandas as pd
import math
from datetime import *
from dateutil import parser
import itertools
from collections import defaultdict
import ast
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
dataset = 'angular.js'

# Process raw data 
a JSON file

In [ ]:
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/'+dataset+'.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'subProject': '', 'changeId': '150346774', 'changeNumber': 16317, 'timestamp': 1509637940000, 'reviewers': [{'accountId': '15655', 'email': '', 'name': 'petebacondarwin', 'avatar': 'https://avatars0.githubusercontent.com/u/15655?v=4'}], 'owner': {'accountId': '1153097', 'email': '', 'name': 'Narretz', 'avatar': 'https://avatars1.githubusercontent.com/u/1153097?v=4'}, 'filePaths': [{'location': 'scripts/travis/travis_retry.sh'}, {'location': 'karma-shared.conf.js'}, {'location': 'scripts/travis/build.sh'}, {'location': 'lib/grunt/utils.js'}]}
{'subProject': '', 'changeId': '150287060', 'changeNumber': 16316, 'timestamp': 1509624804000, 'reviewers': [{'accountId': '8604205', 'email': '', 'name': 'gkalpak', 'avatar': 'https://avatars2.githubusercontent.com/u/8604205?v=4'}], 'owner': {'accountId': '15655', 'email': '', 'name': 'petebacondarwin', 'avatar': 'https://avatars0.githubusercontent.com/u/15655?v=4'}, 'filePaths': [{'location': 'test/ng/locationSpec.js'}, {'location': 'src/ng/loca

In [ ]:
len(data)

6078

In [ ]:
# get an element of the list of data
data[0]

{'changeId': '150346774',
 'changeNumber': 16317,
 'filePaths': [{'location': 'scripts/travis/travis_retry.sh'},
  {'location': 'karma-shared.conf.js'},
  {'location': 'scripts/travis/build.sh'},
  {'location': 'lib/grunt/utils.js'}],
 'owner': {'accountId': '1153097',
  'avatar': 'https://avatars1.githubusercontent.com/u/1153097?v=4',
  'email': '',
  'name': 'Narretz'},
 'reviewers': [{'accountId': '15655',
   'avatar': 'https://avatars0.githubusercontent.com/u/15655?v=4',
   'email': '',
   'name': 'petebacondarwin'}],
 'subProject': '',
 'timestamp': 1509637940000}

In [ ]:
# read a single feature of an element
data[0]['reviewers']

[{'accountId': '15655',
  'avatar': 'https://avatars0.githubusercontent.com/u/15655?v=4',
  'email': '',
  'name': 'petebacondarwin'}]

In [ ]:
# Extract and save desired features from each pull request. The new desired data is new_data
new_data = []

for i in range(0, len(data)):
  elem_dict = {"changeId":data[i]['changeId'], "reviewers":data[i]['reviewers'], "owner":data[i]['owner'], "filePaths":data[i]['filePaths'], "time":data[i]['timestamp']}
  new_data.append(elem_dict)
  # print(new_data[i])

print(len(new_data))

6078


In [ ]:
new_data[0]

{'changeId': '150346774',
 'filePaths': [{'location': 'scripts/travis/travis_retry.sh'},
  {'location': 'karma-shared.conf.js'},
  {'location': 'scripts/travis/build.sh'},
  {'location': 'lib/grunt/utils.js'}],
 'owner': {'accountId': '1153097',
  'avatar': 'https://avatars1.githubusercontent.com/u/1153097?v=4',
  'email': '',
  'name': 'Narretz'},
 'reviewers': [{'accountId': '15655',
   'avatar': 'https://avatars0.githubusercontent.com/u/15655?v=4',
   'email': '',
   'name': 'petebacondarwin'}],
 'time': 1509637940000}

In [ ]:
# Remove remaining undesired features. The final desired information is saved in final_data
final_data = []

for i in range(0, len(new_data)):
  if len(new_data[i]['filePaths']) != 0:
    elem_dict = {"time":new_data[i]['time'], "changeId":new_data[i]['changeId'], "ownerId": new_data[i]['owner']['accountId'], "reviewersId":[], "filePaths":[]}

    for j in range(0, len(new_data[i]['reviewers'])):
      elem_dict["reviewersId"].append(new_data[i]['reviewers'][j]['accountId'])

    for j in range(0, len(new_data[i]['filePaths'])):
      elem_dict["filePaths"].append(new_data[i]['filePaths'][j]['location'])
    
    final_data.append(elem_dict)
    # print(final_data[i])

print(len(final_data))

6022


In [ ]:
final_data[0]

{'changeId': '150346774',
 'filePaths': ['scripts/travis/travis_retry.sh',
  'karma-shared.conf.js',
  'scripts/travis/build.sh',
  'lib/grunt/utils.js'],
 'ownerId': '1153097',
 'reviewersId': ['15655'],
 'time': 1509637940000}

In [ ]:
# change timestamp value from int to date format
for pr in final_data:
  timestamp = int(pr['time']/1000)  # removing the 3 last zero digits 
  dt_obj = datetime.fromtimestamp(timestamp) 
  pr['time'] = dt_obj

final_data[0]

{'changeId': '150346774',
 'filePaths': ['scripts/travis/travis_retry.sh',
  'karma-shared.conf.js',
  'scripts/travis/build.sh',
  'lib/grunt/utils.js'],
 'ownerId': '1153097',
 'reviewersId': ['15655'],
 'time': datetime.datetime(2017, 11, 2, 15, 52, 20)}

In [ ]:
# sort data by time
sorted_data = sorted(final_data, key=lambda d: d['time'])

# Get the Time of available data from selected project
print("Start date:", sorted_data[0]['time'])
print("End date:", sorted_data[-1]['time'])

Start date: 2010-09-17 21:19:39
End date: 2017-11-02 15:52:20


In [ ]:
# use data up to 31 July 2016
sorted_data = [pr for pr in sorted_data if (pr['time'] <= datetime(2016, 7, 31, 23, 59, 59))]
len(sorted_data)

5584

In [ ]:
# Convert list to json
# It should be a string before write it to file
# "default" is added to dumps to make anything not serializable for JSON to string format (like datatime) 
json_string = json.dumps(sorted_data, default=str) 

# Save OpenStack new dataset
file = open(dataset+'_final.json', 'w')
file.write(json_string)
file.close()

In [ ]:
src = "./"+dataset+"_final.json"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/"+dataset+"_final.json"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/angular.js_final.json'

# Implementation

## Load and Clean data

In [ ]:
# Load data from drive
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/'+dataset+'_final.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'time': '2010-09-17 21:19:39', 'changeId': '6023', 'ownerId': '265981', 'reviewersId': ['111951'], 'filePaths': ['src/Resource.js', 'test/ResourceSpec.js']}
{'time': '2010-09-27 04:50:12', 'changeId': '8942', 'ownerId': '216296', 'reviewersId': ['111951'], 'filePaths': ['test/BrowserSpecs.js', 'test/servicesSpec.js', 'src/services.js', 'src/Browser.js', 'test/angular-mocks.js']}
{'time': '2010-10-07 20:39:29', 'changeId': '13139', 'ownerId': '415779', 'reviewersId': ['111951'], 'filePaths': ['test/servicesSpec.js', 'src/services.js', 'test/directivesSpec.js', 'test/ScopeSpec.js', 'src/directives.js', 'src/Scope.js', 'src/Compiler.js']}
{'time': '2010-10-08 16:24:18', 'changeId': '13460', 'ownerId': '46647', 'reviewersId': ['216296'], 'filePaths': ['example/calculator-minified_init.html', 'example/calculator.html', 'example/calculator-bootstrap.html']}
{'time': '2010-10-08 22:22:09', 'changeId': '13602', 'ownerId': '229613', 'reviewersId': ['111951'], 'filePaths': ['test/ParserTest.js'

In [ ]:
# convert times to datetime format
for pr in data:
  pr['time'] = parser.parse(pr['time'])

data[0]

{'changeId': '6023',
 'filePaths': ['src/Resource.js', 'test/ResourceSpec.js'],
 'ownerId': '265981',
 'reviewersId': ['111951'],
 'time': datetime.datetime(2010, 9, 17, 21, 19, 39)}

In [ ]:
len(data)

5584

In [ ]:
# Save only PRs with 1 reviwer (to make the results similar to the paper)
final_data = []
for pr in data:
  if len(pr['reviewersId']) > 1:
    revs = pr['reviewersId']
    for r in revs:
      pr['reviewersId'] = [r]
      final_data.append(pr)
  else:
    final_data.append(pr)

len(final_data)

11073

## Approach

In [ ]:
def get_all_reviewers():
  # create a set of all reviewers
  all_reviewers = set()   
  for pr in final_data:
    for r in pr['reviewersId']:
      all_reviewers.add(r)
  return all_reviewers

all_reviewers = list(get_all_reviewers())
print(all_reviewers)
print(len(all_reviewers))

['673101', '509946', '108273', '367262', '235062', '5173831', '5034942', '580514', '599418', '1507086', '338073', '238841', '828362', '6094917', '37704', '190647', '5419132', '1577682', '83109', '560894', '1152768', '47317', '214998', '127535', '51488', '1271410', '536624', '461827', '5284', '5470108', '7208', '1003261', '3991426', '978310', '95380', '47968', '142800', '6698344', '973550', '2309959', '740240', '656870', '921609', '364416', '216296', '682097', '7169085', '646125', '455656', '4418325', '5513257', '1964439', '242386', '53922', '226421', '2319711', '276898', '5108906', '923677', '52673', '38754', '1764161', '544581', '69736', '2014734', '2697182', '1060689', '1447245', '711926', '1914344', '934004', '224311', '128227', '579922', '46647', '2923269', '260347', '3728897', '5555302', '485368', '715922', '1153097', '2905949', '44419', '94334', '4303', '16091676', '660372', '132584', '132602', '515722', '7273965', '4462396', '3873482', '1066961', '10867000', '912457', '1024005',

In [ ]:
all_reviewers = ['673101', '509946', '108273', '367262', '235062', '5173831', '5034942', '580514', '599418', '1507086', '338073', '238841', '828362', '6094917', '37704', '190647', '5419132', '1577682', '83109', '560894', '1152768', '47317', '214998', '127535', '51488', '1271410', '536624', '461827', '5284', '5470108', '7208', '1003261', '3991426', '978310', '95380', '47968', '142800', '6698344', '973550', '2309959', '740240', '656870', '921609', '364416', '216296', '682097', '7169085', '646125', '455656', '4418325', '5513257', '1964439', '242386', '53922', '226421', '2319711', '276898', '5108906', '923677', '52673', '38754', '1764161', '544581', '69736', '2014734', '2697182', '1060689', '1447245', '711926', '1914344', '934004', '224311', '128227', '579922', '46647', '2923269', '260347', '3728897', '5555302', '485368', '715922', '1153097', '2905949', '44419', '94334', '4303', '16091676', '660372', '132584', '132602', '515722', '7273965', '4462396', '3873482', '1066961', '10867000', '912457', '1024005', '567927', '2840504', '311702', '122831', '656630', '2120258', '9501', '273283', '14321055', '1169263', '506129', '1071917', '1863005', '628508', '469908', '119117', '1540597', '3661051', '6197153', '16632', '445106', '901145', '1840284', '3580270', '1707463', '711371', '365690', '128755', '1203104', '188257', '1227125', '3451581', '309459', '3116399', '1727881', '1308199', '649154', '723723', '1270425', '4841149', '9974', '376367', '7044', '6732159', '3095424', '1331190', '1199742', '497168', '8366322', '74486', '463703', '95672', '6967652', '173559', '627997', '296057', '171858', '4184747', '4390806', '512288', '423822', '1026940', '1418177', '6155484', '455925', '929834', '472917', '136907', '3496424', '621187', '252860', '551022', '7997555', '50030', '391394', '54370', '450708', '1478333', '547387', '1783800', '1792593', '89246', '1728096', '8604205', '1254575', '1985555', '3621643', '5615418', '640797', '882171', '82556', '2043945', '955500', '729275', '93018', '1385752', '4038192', '587826', '140343', '829827', '914697', '3517429', '12898908', '203188', '15655', '1811738', '180963', '76594', '1696510', '814355', '2001903', '521109', '157812', '474988', '127199', '333781', '1238020', '2081709', '1633923', '4602198', '682658', '3760840', '1268018', '989898', '143402', '5136666', '7307099', '146192', '105464', '338488', '3113', '2825653', '10798762', '199499', '16409470', '2219330', '1703387', '953079', '479121', '2294695', '448694', '161889', '3755029', '1541725', '141054', '417674', '10137', '1083817', '1225084', '973915', '1256577', '5558549', '2064558', '168071', '4009570', '5090919', '502633', '1414577', '3888965', '529', '208774', '690815', '1113607', '4871841', '1716267', '987389', '210413', '588473', '147919', '4506749', '261503', '10366310', '637449', '616192', '4027358', '224732', '1148376', '4082216', '445208', '1771122', '1246630', '108938', '2320694', '1389011', '312855', '1157931', '3497122', '313893', '149541', '1533589', '910753', '139133', '821510', '28537', '1476792', '1526680', '4668335', '1045233', '227746', '411852', '10065080', '12800', '139095', '1979087', '307987', '217285', '4222494', '502412', '16798078', '6394528', '4078946', '220039', '7551644', '894198', '793037', '9009611', '46198', '953700', '5089377', '774864', '87463', '1758366', '2053478', '3302176', '2685357', '981154', '422841', '888070', '4673273', '824762', '1198615', '111951', '976608', '1163554', '5147329', '4841675', '1442897']

print("All Reviewers in project:", all_reviewers)
print("Number of Reviewers in whole project:", len(all_reviewers))

All Reviewers in project: ['673101', '509946', '108273', '367262', '235062', '5173831', '5034942', '580514', '599418', '1507086', '338073', '238841', '828362', '6094917', '37704', '190647', '5419132', '1577682', '83109', '560894', '1152768', '47317', '214998', '127535', '51488', '1271410', '536624', '461827', '5284', '5470108', '7208', '1003261', '3991426', '978310', '95380', '47968', '142800', '6698344', '973550', '2309959', '740240', '656870', '921609', '364416', '216296', '682097', '7169085', '646125', '455656', '4418325', '5513257', '1964439', '242386', '53922', '226421', '2319711', '276898', '5108906', '923677', '52673', '38754', '1764161', '544581', '69736', '2014734', '2697182', '1060689', '1447245', '711926', '1914344', '934004', '224311', '128227', '579922', '46647', '2923269', '260347', '3728897', '5555302', '485368', '715922', '1153097', '2905949', '44419', '94334', '4303', '16091676', '660372', '132584', '132602', '515722', '7273965', '4462396', '3873482', '1066961', '10867

In [ ]:
# problem hyper-parameters
lambda_val = 7  # can be changed
gama = 0.7

### Time-decay Relationship

In [ ]:
def find_PRlist(pull_reqs, reviewerId, cur_time):

  # lambda days before the time of our test PR
  lambda_ago = cur_time - timedelta(days=lambda_val)
  one_day_ago = cur_time - timedelta(days=1)

  # list of PRs in "pull_reqs" that are between time of lambda days ago up to the time of test PR
  PS = [pr for pr in pull_reqs if ((lambda_ago < pr['time'] < one_day_ago) and (reviewerId in pr['reviewersId']))]
  # print("size of final list:", len(PS))
  # print(PS)

  return PS 

In [ ]:
def cal_decayRelation(pull_reqs, cur_time):
  return sum([(1/(cur_time-pr['time']).days) for pr in pull_reqs])

### Time-decay File Similarity

In [ ]:
def pathDistance(f1, f2):
  
  commen_prefix = os.path.commonprefix([f1, f2]).split("/")
  commen_len = len(commen_prefix)
  if '' in commen_prefix:
   commen_len -= 1
  # print(commen_prefix, commen_len)

  max_len = max(len(f1.split("/")), len(f2.split("/")))
  # print(max_len)

  return commen_len / max_len

In [ ]:
def prDistance(pr1, pr2):
  return sum([pathDistance(f1, f2) for f1 in pr1['filePaths'] for f2 in pr2['filePaths']]) / (len(pr1['filePaths'])*len(pr2['filePaths']))

In [ ]:
def cal_decayFileSimilarity(pull_reqs, cur_time, test_pr):
  return sum([prDistance(test_pr, pr)*(1/(cur_time-pr['time']).days) for pr in pull_reqs])

### Feature Extraction
Construct a Pull Request Vector for each PR

In [ ]:
# For all PRs we should extract PRvector and save this and reviewer ID into a dataframe for future use

vectors = []
labels = []

for t in range(0, len(final_data)):

  prev_pulls = final_data[0:t]
  cur_pull = final_data[t]

  # list of PRs that submitted by the owner of this PR in trainset
  OS = [pr for pr in prev_pulls if (pr['ownerId'] == cur_pull['ownerId'])]

  cur_time = cur_pull['time']

  # Calculating time-decaying Relationship
  PRvector = [cal_decayRelation(find_PRlist(OS, rev, cur_time), cur_time) for rev in all_reviewers]

  # Calculating time-decaying File Similarity
  PRvector.extend([cal_decayFileSimilarity(find_PRlist(prev_pulls, rev, cur_time), cur_time, cur_pull) for rev in all_reviewers])

  if t % 200 == 0:
    print("PR Vector #", t, ": ", PRvector)

  # Save to dataframe
  vectors.append(PRvector)
  labels.append(cur_pull['reviewersId'][0])


print("PR vector size:", len(all_reviewers), "is equal to: 2 *", len(all_reviewers), "=", 2*len(all_reviewers))
print(len(vectors), len(labels))

PR Vector # 0 :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# Create a dataframe of vectors and labels(reviewerIDs)
df = pd.DataFrame()
df['PRvector'] = vectors
df['reviewerID'] = labels
df.head()

,PRvector,reviewerID
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",216296
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951


In [ ]:
# Save dataframe as csv file
df.to_csv('dataframe.csv')

In [ ]:
# Save dataframe into Drive
src = "./dataframe.csv"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/"+dataset+"_FeatureVectors.csv"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/angular.js_FeatureVectors.csv'

### Experiment & Result

In [ ]:
def pred_classes(index):
  all_reviewers = set()   
  for pr in final_data[0:index]:
    for r in pr['reviewersId']:
      all_reviewers.add(int(r))

  return sorted(all_reviewers)

In [ ]:
# Function to write predicted top-k reviewers for each test into a JSON file
def save_predictions(pred_list):
  with open('output.json', 'a') as outfile:
    json.dump(pred_list, outfile, indent=4)

In [ ]:
def create_pred_obj(group_num, test_num, pred_revs, actual_rev):
  pred_obj = {}
  pred_obj['changeId'] = str(group_num) + "g" + str(test_num) 
  pred_obj['actual'] = [actual_rev]
  pred_obj['recommended'] = pred_revs
  return pred_obj

### Main

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project/'+dataset+'_FeatureVectors.csv')
df.drop('Unnamed: 0', inplace=True, axis=1)
df.head()

,PRvector,reviewerID
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",216296
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",111951


In [ ]:
# PRs created in the same month should be in the same group
groups = []

start_year = final_data[0]['time'].year
start_month = final_data[0]['time'].month
group = []

for i in range(0, len(final_data)):
  if (final_data[i]['time'].year == start_year) and (final_data[i]['time'].month == start_month):
    group.append(df.loc[i, :])
    if(i == len(final_data)-1): # for the last PR
      groups.append(group)
  else:
    groups.append(group)
    group = []
    start_year = final_data[i]['time'].year
    start_month = final_data[i]['time'].month
    group.append(df.loc[i, :])

len(groups)

71

In [ ]:
rounds_num = len(groups)-1

RFmodel = RandomForestClassifier(n_estimators=12, max_depth=None, random_state=7)
NBmodel = GaussianNB()

train = groups[0]
# setting X, y to use for model construction
X = [ast.literal_eval(train[j]["PRvector"]) for j in range(0, len(train))]
y = [train[j]["reviewerID"] for j in range(0, len(train))]

for i in range(0, rounds_num):

  predictions_list = []  # final predicted reviewers for each test PR

  print("********** Test Group number", i+1, "**********")

  test = groups[i+1]
  X_test = [ast.literal_eval(test[j]["PRvector"]) for j in range(0, len(test))]
  y_test = [test[j]["reviewerID"] for j in range(0, len(test))]

  # use classifiers to find probabilities for each test
  for t in range(0, len(X_test)):

    # Random Forest classifier
    RFmodel.fit(X, y)
    RF_prob = RFmodel.predict_proba(np.reshape(X_test[t], (1, -1)))

    # Naive Bayes classifier
    NBmodel.fit(X, y)
    NB_prob = NBmodel.predict_proba(np.reshape(X_test[t], (1, -1)))
    
    # Composer
    comp_prob = (RF_prob * gama) + (NB_prob * (1-gama))
    comp_prob = [round(c, 2) for c in comp_prob[0]]

    # dictionary of each label and its probability
    results = dict(zip(pred_classes(len(X)), comp_prob))  
    # sort predicted reviewers by probability values
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)  
    # extract maximum 10 first predictions
    pred_revs = [str(tup[0]) for tup in sorted_results[0:10] if (tup[1]!=0)]

    if t % 10 == 0:
      print("Test number", t)
      print("Predicted reviewers:", pred_revs)
      print("Actual reviewer:", str(y_test[t]))
      print("-----------------")

    # make object of real and predicted reviewers
    predictions_list.append(create_pred_obj(i+1, t, pred_revs, str(y_test[t])))

    # append the test PR to trainset
    X.append(X_test[t])
    y.append(y_test[t])


  # save objects into a file
  save_predictions(predictions_list)


********** Test Group number 1 **********
Test number 0
Predicted reviewers: ['111951']
Actual reviewer: 111951
-----------------


/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:489: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:490: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:489: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:490: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:489: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.

Test number 10
Predicted reviewers: ['216296', '111951']
Actual reviewer: 46647
-----------------
********** Test Group number 2 **********
Test number 0
Predicted reviewers: ['216296', '111951']
Actual reviewer: 216296
-----------------
Test number 10
Predicted reviewers: ['111951', '216296']
Actual reviewer: 111951
-----------------
********** Test Group number 3 **********
Test number 0
Predicted reviewers: ['111951', '216296']
Actual reviewer: 216296
-----------------
********** Test Group number 4 **********
Test number 0
Predicted reviewers: ['474988', '111951', '216296']
Actual reviewer: 111951
-----------------
Test number 10
Predicted reviewers: ['216296', '46647', '111951']
Actual reviewer: 111951
-----------------
********** Test Group number 5 **********
Test number 0
Predicted reviewers: ['216296', '111951']
Actual reviewer: 216296
-----------------
Test number 10
Predicted reviewers: ['216296']
Actual reviewer: 216296
-----------------
********** Test Group number 6 *****